In [1]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT
import random

from tools.classical import *
from tools.quantum import *

# https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/quantum_phase_estimation.ipynb

In [2]:
num_bits_estimate = 4
# For 2x2 matrix one qubit is enough
q = QuantumRegister(1, name="q")
# In QPE we use n ancillas to estimate n bits from the phase
a = QuantumRegister(num_bits_estimate, name="a") 
# For n ancillary qubit measurment we need n cllasical bits
c = ClassicalRegister(num_bits_estimate, name="c") # Create a quantum circuit
circuit = QuantumCircuit(a, q, c)

# |1> eigenstate initialization
circuit.x(q[0])

In [3]:
random.seed(667)
E_1, E_2 = (random.random(), random.random())
print("We are going to estimate E_2 via QPE algorithm \nE_2 = {}".format(E_2))

# circuit for unitary operator U = exp(iHt)
t = 1
unitary = QuantumCircuit(q, name="U")

unitary.p(2*np.pi*E_2 * t, q[0]) # q[0] is the only qubit in q register
unitary.x(q[0])
unitary.p(2*np.pi*E_1 * t, q[0])
unitary.x(q[0])

cU = unitary.control(1)

We are going to estimate E_2 via QPE algorithm 
E_2 = 0.4343665887494911


In [4]:
for anc in a:
    circuit.h(anc)

for n in range(a.size):
    for m in range(2**n):
        circuit.compose(cU, qubits=[a[n], q[0]], inplace=True)

qft_circ = QFT(num_bits_estimate, do_swaps=True, inverse=True)  #! SWAPS=True
circuit.compose(qft_circ, a, inplace=True)
circuit.measure(a, c)

tr_circuit = transpile(circuit, basis_gates=['u3', 'cx'], optimization_level=3)

/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [5]:
backend = Aer.get_backend('qasm_simulator')
shots = 128  # how many time execute the algorithm
job = backend.run(tr_circuit, shots=shots)
result = job.result()
counts = result.get_counts()
print(counts)
phase_bits = max(counts, key=counts.get) # take the most often obtaned result
print(f'Max count was for phase bits {phase_bits} : {counts[phase_bits]} times')

{'0111': 128}
Max count was for phase bits 0111 : 128 times


In [7]:
phase = 0
for index, bit in enumerate((phase_bits)):  #! Bits are already in order
    phase += int(bit) / 2**(index + 1)
    
estimated_E_2 = phase / t
print("Eigenvalue of the Hamiltonian: {}".format(E_2))
print("Estimated eigenvalue of the Hamiltonian: {}".format(estimated_E_2))

Eigenvalue of the Hamiltonian: 0.4343665887494911
Estimated eigenvalue of the Hamiltonian: 0.4375
